## Etapa atual
- **Validação da ideia**: criação de MVP manual para comparação de preços.  
- **Criação de base inicial**: coletar dados de algumas notas fiscais ou produtos.  
- **Objetivo desta etapa**: ter **uma base inicial de preços** para análises e comparativos.

##  Estrutura para etapa atual
1. Criar **script para raspar dados das notas fiscais**:  
   - Extrair: produto, preço, mercado, data.  
2. Alimentar **planilha ou banco de dados simples** (CSV / Google Sheets / SQLite).  
3. Fazer **comparativo de preços iniciais** usando esses dados.  
4. Validar se o MVP manual fornece resultados confiáveis antes de automatizar scraping de sites ou QR Codes.


In [6]:
import requests
from bs4 import BeautifulSoup

In [21]:
response = requests.get("https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651020000029821102030260|2|1|1|7CEABA67901306C7669AF902F102EC944F682413")

html = response.text

soup = BeautifulSoup(html, 'html.parser')

In [22]:
paragrafos = soup.find_all('p')

In [28]:
div_id = soup.find('div', id='u20')

In [29]:
div_id

<div class="txtTopo" id="u20">MERCADO JARDIM ZAIRA LTDA</div>

In [71]:
nome_produto = soup.find_all('span', class_='txtTit')

print(nome_produto)

[<span class="txtTit">REQUEIJAO TIROLEZ  CREMOSO 200GR</span>, <span class="txtTit">FP PAO LEITINHO KG</span>, <span class="txtTit">CHIPA MINNAS KG</span>, <span class="txtTit">FP PAO LEITE KG</span>, <span class="txtTit">PAO QUEIJO SABOR  MINAS 1KG</span>, <span class="txtTit">MORTADELA SEARA DEFUMADA KG BANDEJA</span>]


In [75]:
nomes_produtos = [p.get_text(strip=True) for p in nome_produto]


In [76]:
nomes_produtos

['REQUEIJAO TIROLEZ  CREMOSO 200GR',
 'FP PAO LEITINHO KG',
 'CHIPA MINNAS KG',
 'FP PAO LEITE KG',
 'PAO QUEIJO SABOR  MINAS 1KG',
 'MORTADELA SEARA DEFUMADA KG BANDEJA']

In [81]:
quantidade = soup.find_all('span', class_='Rqtd')

quantidade_comprada = [p.get_text(strip=True) for p in quantidade]
quantidade_comprada 

['Qtde.:1',
 'Qtde.:0,276',
 'Qtde.:0,248',
 'Qtde.:0,138',
 'Qtde.:1',
 'Qtde.:0,12']

In [83]:
unidade = soup.find_all('span', class_='RUN')

unidade_produto = [p.get_text(strip=True)for p in unidade]
unidade_produto

['UN:UN0001', 'UN:KG0001', 'UN:KG0001', 'UN:KG0001', 'UN:UN0001', 'UN:KG0001']

In [85]:
valor_unitario = soup.find_all('span',class_='RvlUnit')

vl_unitario = [p.get_text(strip=True) for p in valor_unitario]
vl_unitario

['Vl. Unit.:11,99',
 'Vl. Unit.:39,891304',
 'Vl. Unit.:39,919355',
 'Vl. Unit.:19,927536',
 'Vl. Unit.:19,9',
 'Vl. Unit.:28,916667']

In [90]:
valor_total = soup.find_all('td',class_='txtTit noWrap')

vl_total = [p.get_text(strip=True) for p in valor_total]

vl_total

['Vl. Total11,99',
 'Vl. Total11,01',
 'Vl. Total9,90',
 'Vl. Total2,75',
 'Vl. Total19,90',
 'Vl. Total3,47']